### PA Model Data Preprocessing

In [9]:
# packages
import pandas as pd
import numpy as np
import os
os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/MLB Season Model/') 

In [10]:
data_full = pd.read_csv('./data/cleaned/data_15_24.csv')
injury_df = pd.read_csv('./data/full/injury_data.csv')

#### Injury Data

Filter out Pitchers, Drop Unneeded Cols, Fill Return Date NAs

In [11]:
injury_df = injury_df[~injury_df['Pos'].isin(['SP', 'RP', 'SP/RP'])]
injury_df = injury_df.drop(columns=['Pos', 'Injury / Surgery Date', 'Status' , 'Latest Update', 'Eligible to Return'])
injury_df.loc[injury_df['Return Date'].isna(), 'Return Date'] = '10/03/' + injury_df['Season'].astype(str) 
# on average last day of season. It differs like 2-3 days a year but this makes the code alot simpler lol

Clean Names

In [12]:
injury_df = injury_df.rename(columns={'NameASCII': 'Name'})
injury_df['Name'] = injury_df['Name'].str.lower()
injury_df['Name'] = injury_df['Name'].str.replace(r'[^a-z\s]', '', regex=True)

Date Col to Datetime Obj

In [13]:
injury_df['IL Retro Date'] = pd.to_datetime(injury_df['IL Retro Date']).dt.dayofyear
injury_df['Return Date'] = pd.to_datetime(injury_df['Return Date']).dt.dayofyear

Days Missed Each Season

In [14]:
injury_df['Days_Missed'] = injury_df['Return Date'] - injury_df['IL Retro Date']
injury_df = injury_df.drop(columns=['Team', 'Injury / Surgery', 'IL Retro Date', 'Return Date'])
injury_df['Days_Missed_Season'] = injury_df.groupby(['Name', 'Season'])['Days_Missed'].transform('sum')
injury_df = injury_df.drop(columns=['Days_Missed'])
injury_df = injury_df.drop_duplicates()

#### match to PA df

In [15]:
pa_data = data_full[['Name', 'Season', 'PA', 'Age']]
pa_data = pa_data.reset_index()
pa_data = pa_data.drop(columns=['index'])
pa_data = pa_data[pa_data['Season'] > 2020]
pa_data = pa_data.merge(injury_df, how='outer', on=['Name', 'Season'])

In [16]:
pa_data['Days_Missed_Season'] = pa_data['Days_Missed_Season'].fillna(0)
pa_data = pa_data.dropna()

Export to CSV

In [17]:
pa_data.to_csv('./data/cleaned/pa_injury_data.csv')